In [1]:
# Import Libraries
import os
import glob
import itertools
import numpy as np
import pandas as pd
from numpy.random import seed, randint

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
# Read CSV and set date as the index column. Parsing date in a format to avoid confusion and speed up parsing.
df = pd.read_csv('..\\..\\data\\processed\\weather\\nrel\\nycNRELwSR.csv', index_col=[0], \
                 parse_dates=True, infer_datetime_format="%m/%d/%Y %H:%M:%S")
df.head()

DHI  DNI  GHI  Clearsky DHI  Clearsky DNI  Clearsky GHI  \
1998-01-01 00:00:00    0    0    0             0             0             0   
1998-01-01 00:30:00    0    0    0             0             0             0   
1998-01-01 01:00:00    0    0    0             0             0             0   
1998-01-01 01:30:00    0    0    0             0             0             0   
1998-01-01 02:00:00    0    0    0             0             0             0   

                     Cloud Type  Dew Point  Solar Zenith Angle  Fill Flag  \
1998-01-01 00:00:00           0      -12.0              162.24          0   
1998-01-01 00:30:00           0      -12.0              161.09          0   
1998-01-01 01:00:00           0      -13.0              158.12          0   
1998-01-01 01:30:00           0      -13.0              153.94          0   
1998-01-01 02:00:00           0      -13.0              149.06          0   

                     ...  Elevation Angle (in deg)  Azimuth Angle (in deg)  \
1998-01-01 00:00:00  ...                    -72.24                    0.40   
1998-01-01 00:30:00  ...                    -72.24                    0.40   
1998-01-01 01:00:00  ...                    -68.15                   40.22   
1998-01-01 01:30:00  ...                    -68.15                   40.22   
1998-01-01 02:00:00  ...                    -59.11                   64.16   

                     SR_N  SR_S  SR_E  SR_W  SR_NE  SR_SE  SR_NW  SR_SW  
1998-01-01 00:00:00   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0  
1998-01-01 00:30:00   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0  
1998-01-01 01:00:00   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0  
1998-01-01 01:30:00   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0  
1998-01-01 02:00:00   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0  

[5 rows x 41 columns]

In [3]:
# Columns within the dataframe
df.columns

Index(['DHI', 'DNI', 'GHI', 'Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI',
       'Cloud Type', 'Dew Point', 'Solar Zenith Angle', 'Fill Flag',
       'Surface Albedo', 'Wind Speed', 'Precipitable Water', 'Wind Direction',
       'Relative Humidity', 'Temperature', 'Pressure', 'Latitude', 'Longitude',
       'Time Zone', 'Elevation', 'Local Time Zone', 'Day of Year',
       'B-value (in degrees)', 'B-value (in radians)', 'EoT (in minutes)',
       'LSTM (in degrees)', 'TC (in minutes)', 'LST (in hours)',
       'HRA (in degrees)', 'Declination (in degrees)',
       'Elevation Angle (in deg)', 'Azimuth Angle (in deg)', 'SR_N', 'SR_S',
       'SR_E', 'SR_W', 'SR_NE', 'SR_SE', 'SR_NW', 'SR_SW'],
      dtype='object')

In [4]:
# Variations in the data files
listOfOccType = ['BC', 'FwoK', 'FwK', 'SC'] # type of occupants
appDirCols = ['SR_N', 'SR_S', 'SR_E', 'SR_W', 'SR_NE', 'SR_SE', 'SR_NW', 'SR_SW'] # type of apartment directions
routineVariation = ['FR', 'DR'] # type of routines
# setPointVariation = ['SR', 'WbR', 'RR'] # type of set point variations

In [5]:
# Subset data for machine learning framework
baseCols = ['Temperature', 'Dew Point', 'Relative Humidity']

# Data columns in the machine learning framework
frameworkCols = baseCols + appDirCols

# Subset the data framework
df_subset = df[frameworkCols].copy()
df_subset.head()

Temperature  Dew Point  Relative Humidity  SR_N  SR_S  \
1998-01-01 00:00:00         -9.0      -12.0              78.77   0.0   0.0   
1998-01-01 00:30:00         -9.0      -12.0              78.81   0.0   0.0   
1998-01-01 01:00:00        -10.0      -13.0              76.14   0.0   0.0   
1998-01-01 01:30:00        -10.0      -13.0              76.16   0.0   0.0   
1998-01-01 02:00:00        -10.0      -13.0              80.40   0.0   0.0   

                     SR_E  SR_W  SR_NE  SR_SE  SR_NW  SR_SW  
1998-01-01 00:00:00   0.0   0.0    0.0    0.0    0.0    0.0  
1998-01-01 00:30:00   0.0   0.0    0.0    0.0    0.0    0.0  
1998-01-01 01:00:00   0.0   0.0    0.0    0.0    0.0    0.0  
1998-01-01 01:30:00   0.0   0.0    0.0    0.0    0.0    0.0  
1998-01-01 02:00:00   0.0   0.0    0.0    0.0    0.0    0.0

In [6]:
# Convert temperature and dewpoint from Celcius to Fahrenheit 
df_subset["Temperature"] = (df_subset["Temperature"] * 9 / 5) + 32
df_subset["Dew Point"] = (df_subset["Dew Point"] * 9 / 5) + 32
df_subset["Month"] = df_subset.index.map(lambda x: x.month)

df_subset.head()

Temperature  Dew Point  Relative Humidity  SR_N  SR_S  \
1998-01-01 00:00:00         15.8       10.4              78.77   0.0   0.0   
1998-01-01 00:30:00         15.8       10.4              78.81   0.0   0.0   
1998-01-01 01:00:00         14.0        8.6              76.14   0.0   0.0   
1998-01-01 01:30:00         14.0        8.6              76.16   0.0   0.0   
1998-01-01 02:00:00         14.0        8.6              80.40   0.0   0.0   

                     SR_E  SR_W  SR_NE  SR_SE  SR_NW  SR_SW  Month  
1998-01-01 00:00:00   0.0   0.0    0.0    0.0    0.0    0.0      1  
1998-01-01 00:30:00   0.0   0.0    0.0    0.0    0.0    0.0      1  
1998-01-01 01:00:00   0.0   0.0    0.0    0.0    0.0    0.0      1  
1998-01-01 01:30:00   0.0   0.0    0.0    0.0    0.0    0.0      1  
1998-01-01 02:00:00   0.0   0.0    0.0    0.0    0.0    0.0      1

In [7]:
configParams = {
                'Cold':{'Clothing': 0.78, 
                        'BC': {'SPT': {'Mi':68, 'Mid':71, 'Ma':75}, 'ActivityLevel': 1.3},
                        'FwoK': {'SPT': {'Mi':70, 'Mid':66, 'Ma':71}, 'ActivityLevel': 1.7},
                        'FwK': {'SPT': {'Mi':61, 'Mid':62, 'Ma':68}, 'ActivityLevel': 2.0},
                        'SC': {'SPT': {'Mi':72, 'Mid':75, 'Ma':78}, 'ActivityLevel': 1.0}
                       },
                'Cool':{'Clothing': 0.63, 
                        'BC': {'SPT': {'Mi':70, 'Mid':73, 'Ma':77}, 'ActivityLevel': 1.3},
                        'FwoK': {'SPT': {'Mi':64, 'Mid':68, 'Ma':73}, 'ActivityLevel': 1.7},
                        'FwK': {'SPT': {'Mi':60, 'Mid':65, 'Ma':70}, 'ActivityLevel': 2.0},
                        'SC': {'SPT': {'Mi':75, 'Mid':77, 'Ma':80}, 'ActivityLevel': 1.0}
                       },
                'Warm':{'Clothing': 0.56, 
                        'BC': {'SPT': {'Mi':71, 'Mid':74, 'Ma':78}, 'ActivityLevel': 1.3},
                        'FwoK': {'SPT': {'Mi':65, 'Mid':69, 'Ma':73}, 'ActivityLevel': 1.7},
                        'FwK': {'SPT': {'Mi':61, 'Mid':66, 'Ma':71}, 'ActivityLevel': 2.0},
                        'SC': {'SPT': {'Mi':76, 'Mid':77, 'Ma':80}, 'ActivityLevel': 1.0}
                       },
                'Hot':{'Clothing': 0.5, 
                        'BC': {'SPT': {'Mi':72, 'Mid':75, 'Ma':78}, 'ActivityLevel': 1.3},
                        'FwoK': {'SPT': {'Mi':66, 'Mid':70, 'Ma':74}, 'ActivityLevel': 1.7},
                        'FwK': {'SPT': {'Mi':62, 'Mid':67, 'Ma':72}, 'ActivityLevel': 2.0},
                        'SC': {'SPT': {'Mi':77, 'Mid':78, 'Ma':81}, 'ActivityLevel': 1.0}
                       }
                }

In [8]:
for element in itertools.product(listOfOccType, appDirCols, routineVariation):
    
    df_subset["SPT_Mi"] = 70
    df_subset["SPT"] = 70
    df_subset["SPT_Ma"] = 70
    df_subset["SPT_Random"] = 70
    
    seed(1)

    # Recommendations for Cold (Jan, Feb, Dec)
    df_subset.at[df_subset.Month.isin([1, 2, 12]), "Clothing"] = configParams['Cold']['Clothing']
    df_subset.at[df_subset.Month.isin([1, 2, 12]), "SPT_Mi"] = configParams['Cold'][element[0]]['SPT']['Mi']
    df_subset.at[df_subset.Month.isin([1, 2, 12]), "SPT"] = configParams['Cold'][element[0]]['SPT']['Mid']
    df_subset.at[df_subset.Month.isin([1, 2, 12]), "SPT_Ma"] = configParams['Cold'][element[0]]['SPT']['Ma']
    df_subset.at[df_subset.Month.isin([1, 2, 12]), "SPT_Random"] = randint(configParams['Cold'][element[0]]['SPT']['Mi'], \
                                                                            configParams['Cold'][element[0]]['SPT']['Ma'], \
                                                                            df_subset.loc[df_subset.Month.isin([1, 2, 12])].shape[0])
    
    # Recommendations for Cool (Mar, Nov)
    df_subset.at[df_subset.Month.isin([3, 11]), "Clothing"] = configParams['Cool']['Clothing']
    df_subset.at[df_subset.Month.isin([3, 11]), "SPT_Mi"] = configParams['Cool'][element[0]]['SPT']['Mi']
    df_subset.at[df_subset.Month.isin([3, 11]), "SPT"] = configParams['Cool'][element[0]]['SPT']['Mid']
    df_subset.at[df_subset.Month.isin([3, 11]), "SPT_Ma"] = configParams['Cool'][element[0]]['SPT']['Ma']
    df_subset.at[df_subset.Month.isin([3, 11]), "SPT_Random"] = randint(configParams['Cool'][element[0]]['SPT']['Mi'], \
                                                                        configParams['Cool'][element[0]]['SPT']['Ma'], \
                                                                        df_subset.loc[df_subset.Month.isin([3, 11])].shape[0])
    # Recommendations for Warm (Apr, May, Oct)
    df_subset.at[df_subset.Month.isin([4, 5, 10]), "Clothing"] = configParams['Warm']['Clothing']
    df_subset.at[df_subset.Month.isin([4, 5, 10]), "SPT_Mi"] = configParams['Warm'][element[0]]['SPT']['Mi']
    df_subset.at[df_subset.Month.isin([4, 5, 10]), "SPT"] = configParams['Warm'][element[0]]['SPT']['Mid']
    df_subset.at[df_subset.Month.isin([4, 5, 10]), "SPT_Ma"] = configParams['Warm'][element[0]]['SPT']['Ma']
    df_subset.at[df_subset.Month.isin([4, 5, 10]), "SPT_Random"] = randint(configParams['Warm'][element[0]]['SPT']['Mi'], \
                                                                           configParams['Warm'][element[0]]['SPT']['Ma'], \
                                                                           df_subset.loc[df_subset.Month.isin([4, 5, 10])].shape[0])
    # Recommendations for Hot (Jun, Jul, Aug, Sep)
    df_subset.at[df_subset.Month.isin([6, 7, 8, 9]), "Clothing"] = configParams['Hot']['Clothing']
    df_subset.at[df_subset.Month.isin([6, 7, 8, 9]), "SPT_Mi"] = configParams['Hot'][element[0]]['SPT']['Mi']
    df_subset.at[df_subset.Month.isin([6, 7, 8, 9]), "SPT"] = configParams['Hot'][element[0]]['SPT']['Mid']
    df_subset.at[df_subset.Month.isin([6, 7, 8, 9]), "SPT_Ma"] = configParams['Hot'][element[0]]['SPT']['Ma']
    df_subset.at[df_subset.Month.isin([6, 7, 8, 9]), "SPT_Random"] = randint(configParams['Hot'][element[0]]['SPT']['Mi'], \
                                                                             configParams['Hot'][element[0]]['SPT']['Ma'], \
                                                                             df_subset.loc[df_subset.Month.isin([6, 7, 8, 9])].shape[0])
    
    df_subset["ActivityLevel"] = configParams['Cold'][element[0]]['ActivityLevel']
    
    df_subset["ma_diff"] = df_subset["SPT_Ma"].values - df_subset["SPT"].values
    df_subset["mi_diff"] = df_subset["SPT_Mi"].values - df_subset["SPT"].values

    df_subset["temp_diff"] = df_subset["SPT"].values - df_subset["Temperature"].values
    df_subset.loc[df_subset["temp_diff"] >= 0, "adjustedSPT"] = df_subset.loc[df_subset["temp_diff"] >= 0, "SPT"] + \
                                                                 df_subset.loc[df_subset["temp_diff"] >= 0, ["ma_diff", "temp_diff"]].min(axis=1)
    df_subset.loc[df_subset["temp_diff"] < 0, "adjustedSPT"] = df_subset.loc[df_subset["temp_diff"] < 0, "SPT"] + \
                                                                df_subset.loc[df_subset["temp_diff"] < 0, ["mi_diff", "temp_diff"]].max(axis=1)
    
    df_subset["Occupancy"] = 1
    if element[2] == 'FR':
        df_subset.at[df_subset.between_time('09:00', '18:00').index, "Occupancy"] = 0
        df_subset.at[df_subset.index.weekday.isin([5, 6]), "Occupancy"] = 1
    elif element[2] == 'DR':
        df_subset["Occupancy"] = randint(0, 2, df_subset.shape[0])
    
    df_output = df_subset[baseCols + ['Month', "ActivityLevel", "Clothing", element[1], "SPT_Mi", "SPT", "SPT_Ma", "adjustedSPT", "SPT_Random", "Occupancy"]].copy()
    
    df_output.rename(columns={element[1]:'SR'}, inplace=True)
    df_output.to_csv('../../data/processed/framework/d2/apt_' + element[0] + '_' + element[1].split('_')[1] + '_' + element[2] + '.csv')

In [9]:
df_output.head()

Temperature  Dew Point  Relative Humidity  Month  \
1998-01-01 00:00:00         15.8       10.4              78.77      1   
1998-01-01 00:30:00         15.8       10.4              78.81      1   
1998-01-01 01:00:00         14.0        8.6              76.14      1   
1998-01-01 01:30:00         14.0        8.6              76.16      1   
1998-01-01 02:00:00         14.0        8.6              80.40      1   

                     ActivityLevel  Clothing   SR  SPT_Mi  SPT  SPT_Ma  \
1998-01-01 00:00:00            1.0      0.78  0.0      72   75      78   
1998-01-01 00:30:00            1.0      0.78  0.0      72   75      78   
1998-01-01 01:00:00            1.0      0.78  0.0      72   75      78   
1998-01-01 01:30:00            1.0      0.78  0.0      72   75      78   
1998-01-01 02:00:00            1.0      0.78  0.0      72   75      78   

                     adjustedSPT  SPT_Random  Occupancy  
1998-01-01 00:00:00         78.0          77          1  
1998-01-01 00:30:00         78.0          75          0  
1998-01-01 01:00:00         78.0          76          1  
1998-01-01 01:30:00         78.0          72          1  
1998-01-01 02:00:00         78.0          73          1

In [10]:
#df_output["ma_diff"] = df_output["SPT_Ma"].values - df_output["SPT"].values
#df_output["mi_diff"] = df_output["SPT_Mi"].values - df_output["SPT"].values

#df_output["temp_diff"] = df_output["SPT"].values - df_output["Temperature"].values
#df_output.loc[df_output["temp_diff"] >= 0, "adjustedSPT1"] = df_output.loc[df_output["temp_diff"] >= 0, "SPT"] + \
#                                                             df_output.loc[df_output["temp_diff"] >= 0, ["ma_diff", "temp_diff"]].min(axis=1)
#df_output.loc[df_output["temp_diff"] < 0, "adjustedSPT1"] = df_output.loc[df_output["temp_diff"] < 0, "SPT"] + \
#                                                            df_output.loc[df_output["temp_diff"] < 0, ["mi_diff", "temp_diff"]].max(axis=1)

In [11]:
#df_output["adjustedSPT2"] = df_output["SPT"] + \
#                            df_output.apply(lambda x: min(x["ma_diff"], x["temp_diff"]) if x["SPT"] >= x["Temperature"] else max(x["mi_diff"], x["temp_diff"]), axis=1)

In [12]:
#df_output.head()